<a href="https://colab.research.google.com/github/jetsonai/HK_LSTMSenfuClass/blob/main/Day1/CNN/%5B1%5DVGG_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# VGGNet for Color Images (RGB, CIFAR-10)
## TensorFlow/Keras version

In [ ]:
import os, random, math
import numpy as np
import tensorflow as tf
from dataclasses import dataclass

## 1) Reproducibility & Device

In [ ]:
def fix_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
fix_seed(42)


In [ ]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # set memory growth instead of pre-allocating all VRAM
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"✅ GPU detected: {tf.config.experimental.get_device_details(gpus[0]).get('device_name','GPU')}")
    except Exception as e:
        print("⚠️ Could not set memory growth:", e)
else:
    print("⚠️ No GPU detected. Using CPU.")

✅ GPU detected: Tesla T4


## 2) Hyper-params & constants
 #####   ( use_bn_dropout: bool = True 로 변경하면 bn_dropout 예제, use_augmentation: bool = True 로 변경하면 augmentation 예제 )

In [ ]:
@dataclass
class CFG:
    img_size: int = 32
    num_classes: int = 10
    width: int = 16           # base width (like PyTorch code)
    p_dropout: float = 0.25   # for BN/Dropout variant
    batch_size: int = 64
    epochs: int = 50
    lr: float = 1e-2
    use_bn_dropout: bool = True   # toggle VGG16withBN+Dropout variant
    use_augmentation: bool = False
    out_dir: str = "ckpt_tf/vgg16_rgb_bn"  # folder will be renamed based on flag

In [ ]:
CFG = CFG()
CFG.out_dir = f"ckpt_tf/{'vgg16_rgb_bn' if CFG.use_bn_dropout else 'vgg16_rgb'}"
os.makedirs(CFG.out_dir, exist_ok=True)

In [ ]:
# CIFAR-10 mean/std in RGB order (same as PyTorch version)
CIFAR10_MEAN = np.array([0.4914, 0.4822, 0.4465], dtype=np.float32)
CIFAR10_STD  = np.array([0.2470, 0.2435, 0.2616], dtype=np.float32)

## 3) Data: CIFAR-10 (RGB)

In [ ]:
def get_cifar10_datasets(use_augmentation=True):
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
    y_train = y_train.squeeze()
    y_test = y_test.squeeze()
    x_train = x_train.astype(np.float32) / 255.0
    x_test  = x_test.astype(np.float32) / 255.0

    def normalize(img):
        return (img - CIFAR10_MEAN) / CIFAR10_STD

    def augment(img, lbl):
        # --- 추가 옵션 ---
        if use_augmentation:
            img = tf.image.resize_with_crop_or_pad(img, CFG.img_size + 4, CFG.img_size + 4)
            img = tf.image.random_crop(img, size=[CFG.img_size, CFG.img_size, 3])
            img = tf.image.random_flip_left_right(img)
            img = tf.image.random_brightness(img, max_delta=0.1)
            img = tf.image.random_contrast(img, lower=0.8, upper=1.2)
            img = tf.image.random_saturation(img, lower=0.8, upper=1.2)
        img = normalize(img)
        return img, lbl

    def preprocess(img, lbl):
        img = tf.image.resize(img, (CFG.img_size, CFG.img_size))
        img = normalize(img)
        return img, lbl

    train_ds = (
        tf.data.Dataset.from_tensor_slices((x_train, y_train))
        .shuffle(10000, seed=42)
        .map(augment, num_parallel_calls=tf.data.AUTOTUNE)
        .batch(CFG.batch_size)
        .prefetch(tf.data.AUTOTUNE)
    )

    test_ds = (
        tf.data.Dataset.from_tensor_slices((x_test, y_test))
        .map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
        .batch(CFG.batch_size)
        .prefetch(tf.data.AUTOTUNE)
    )

    return train_ds, test_ds

In [ ]:
train_ds, test_ds = get_cifar10_datasets(use_augmentation=CFG.use_augmentation)

## 4) VGG-style building blocks

In [ ]:
from tensorflow.keras import layers, models, regularizers

def ConvBlock(x, out_ch, num_layers=2, use_bn=False):
    for _ in range(num_layers):
        x = layers.Conv2D(out_ch, kernel_size=3, strides=1, padding="same",
                          use_bias=not use_bn,
                          kernel_initializer="he_normal")(x)
        if use_bn:
            x = layers.BatchNormalization()(x)
        x = layers.ReLU()(x)
    return x

In [ ]:
def LinearBlock(x, out_dim, p_drop=0.0, use_dropout=False):
    # PyTorch code uses 3 linear layers ending with num_classes
    # We already have GAP -> (width*4) features; mirror that:
    x = layers.Dense(x.shape[-1], kernel_initializer="he_normal")(x)
    if use_dropout:
        x = layers.Dropout(p_drop)(x)
    x = layers.ReLU()(x)

    x = layers.Dense(x.shape[-1], kernel_initializer="he_normal")(x)
    if use_dropout:
        x = layers.Dropout(p_drop)(x)
    x = layers.ReLU()(x)

    x = layers.Dense(out_dim)(x)
    return x

In [ ]:
def build_vgg16(img_channels=3, width=16, num_classes=10, use_bn_dropout=False, p=0.25):
    inp = layers.Input(shape=(CFG.img_size, CFG.img_size, img_channels))

    # Five conv blocks with 2,2,3,3,3 conv layers (as in the PyTorch version)
    x = ConvBlock(inp, width,     num_layers=2, use_bn=use_bn_dropout)
    x = layers.MaxPool2D(pool_size=2, strides=2)(x)

    x = ConvBlock(x,   width*2,   num_layers=2, use_bn=use_bn_dropout)
    x = layers.MaxPool2D(pool_size=2, strides=2)(x)

    x = ConvBlock(x,   width*4,   num_layers=3, use_bn=use_bn_dropout)
    x = layers.MaxPool2D(pool_size=2, strides=2)(x)

    x = ConvBlock(x,   width*4,   num_layers=3, use_bn=use_bn_dropout)
    x = layers.MaxPool2D(pool_size=2, strides=2)(x)

    x = ConvBlock(x,   width*4,   num_layers=3, use_bn=use_bn_dropout)
    x = layers.MaxPool2D(pool_size=2, strides=2)(x)

    # GAP to mimic AdaptiveAvgPool2d(1).flatten
    x = layers.GlobalAveragePooling2D()(x)

    # Classifier (with optional Dropout, as in VGG16withBN)
    x = LinearBlock(x, num_classes, p_drop=p, use_dropout=use_bn_dropout)

    return models.Model(inp, x, name="VGG16_TF" + ("_BN" if use_bn_dropout else ""))

In [ ]:
model = build_vgg16(img_channels=3,
                    width=CFG.width,
                    num_classes=CFG.num_classes,
                    use_bn_dropout=CFG.use_bn_dropout,
                    p=CFG.p_dropout)

## 5) Compile & Callbacks

In [ ]:
# Using SGD(m=0.9) to match the PyTorch script
opt = tf.keras.optimizers.SGD(learning_rate=CFG.lr, momentum=0.9)

In [ ]:
model.compile(optimizer=opt,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy(name="acc")])

ckpt_best = os.path.join(CFG.out_dir, "best.weights.h5")
ckpt_last = os.path.join(CFG.out_dir, "latest.weights.h5")

In [ ]:
cbs = [
    tf.keras.callbacks.ModelCheckpoint(ckpt_best, monitor="val_acc",
                                       save_best_only=True, save_weights_only=True, verbose=1),
    tf.keras.callbacks.ModelCheckpoint(ckpt_last, monitor="val_acc",
                                       save_best_only=False, save_weights_only=True, verbose=0),
    tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=2, verbose=1),
    tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True, verbose=1)
]


In [ ]:
model.summary()

## 6) Train

In [ ]:
hist = model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=CFG.epochs,
    callbacks=cbs
)

In [ ]:
# Save training curves
import matplotlib.pyplot as plt

plt.figure()
plt.plot(hist.history["loss"], label="train")
plt.plot(hist.history["val_loss"], label="val")
plt.title("Loss (Training vs. Validation)")
plt.xlabel("Epoch"); plt.ylabel("Loss"); plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(CFG.out_dir, "loss.png")); plt.close()

plt.figure()
plt.plot(hist.history["acc"], label="train")
plt.plot(hist.history["val_acc"], label="val")
plt.title("Accuracy (Training vs. Validation)")
plt.xlabel("Epoch"); plt.ylabel("Accuracy"); plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(CFG.out_dir, "accuracy.png")); plt.close()

## 7) Inference helper (evaluate)

In [ ]:
def inference(weights_path: str, batch_size: int = 64):
    """Load weights and evaluate on CIFAR-10 test set."""
    tmp_model = build_vgg16(img_channels=3, width=CFG.width,
                            num_classes=CFG.num_classes,
                            use_bn_dropout=CFG.use_bn_dropout, p=CFG.p_dropout)
    tmp_model.compile(optimizer="sgd",
                      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                      metrics=[tf.keras.metrics.SparseCategoricalAccuracy(name="acc")])
    tmp_model.load_weights(weights_path)
    print(f"Loaded weights from: {weights_path}")
    results = tmp_model.evaluate(test_ds, verbose=1)
    print(f"[Test] Loss: {results[0]:.4f} | Acc.: {results[1]:.4f}")
    return results

In [ ]:
from google.colab import files

def save_trained_model(save_path):
  model.save_weights(save_path)
  print(f"✅ 모델 가중치 저장 완료: {save_path}")

  files.download(save_path)

#### # Example:
inference(ckpt_last)

inference(ckpt_best)
###

In [ ]:
if __name__ == "__main__":
    # quick test: evaluate last checkpoint
    try:
        inference(ckpt_last)
        save_path = "/content/ckpt_tf/vgg16_rgb_final.weights.h5"
        save_trained_model(save_path)

    except Exception as e:
        print("Inference skipped (weights might not be saved yet):", e)